## Benchmark convex hull

In [1]:
import numpy as np
from pympc.geometry.polytope import Polytope
import matplotlib.pyplot as plt

random points

In [2]:
d = 13
n = 50
points = np.random.rand(n, d)

qhull from scipy

In [3]:
from scipy.spatial import ConvexHull
hull = ConvexHull(points)
lhs = hull.equations[:,:-1]
rhs = - hull.equations[:,-1:]
print lhs.shape

(1445038, 13)


In [6]:
from time import time
p = np.random.rand(d,1)
tic = time()
lhs.dot(p)-rhs
print time() - tic

0.149839878082


In [7]:
from pympc.geometry.inner_approximation_polytope_projection import point_in_convex_hull
points_list = [point.reshape(d,1) for point in points]
tic = time()
point_in_convex_hull(p, points_list)
print time() - tic

0.000707149505615
0.0330369472504


remove redundancy with cddlib

p = Polytope(lhs, rhs)
p.assemble()
p.lhs_min.shape

In [ ]:
import cdd
poly_cdd = np.hstack((rhs, -lhs))
mat = cdd.Matrix(poly_cdd, number_type='float')
mat.rep_type = cdd.RepType.INEQUALITY
mat.canonicalize()
lhs = np.zeros((0,d))
rhs = np.zeros((0,1))
for i in range(mat.row_size):
    row = mat.__getitem__(i)
    lhs = np.vstack((lhs, - np.array(row[1:])))
    rhs = np.vstack((rhs, row[0]))
print lhs.shape

cddlib

In [ ]:
import cdd
points_cdd = np.hstack((np.ones((n,1)), points))
mat = cdd.Matrix(points_cdd, number_type='float')
mat.rep_type = cdd.RepType.GENERATOR
poly = cdd.Polyhedron(mat)
inequalities_cdd = poly.get_inequalities()
lhs = np.zeros((0,d))
rhs = np.zeros((0,1))
for i in range(inequalities_cdd.row_size):
    row = inequalities_cdd.__getitem__(i)
    lhs = np.vstack((lhs, - np.array(row[1:])))
    rhs = np.vstack((rhs, row[0]))
print lhs.shape

qhull python wrapper

In [ ]:
from pyhull.convex_hull import ConvexHull
hull = ConvexHull(points)